### 🌡️ Nhiệt độ & độ ẩm

| Tên biến | Giải thích | Đơn vị |
|----------|------------|--------|
| `temp`   | Nhiệt độ không khí trung bình tại bề mặt | °C |
| `dwpt`   | **Điểm sương (dew point)** – nhiệt độ tại đó hơi nước bắt đầu ngưng tụ (có mây, sương) | °C |
| `rhum`   | **Độ ẩm tương đối (relative humidity)** – phần trăm hơi nước trong không khí so với tối đa có thể chứa được | % |

📌 *Điểm sương và độ ẩm thường dùng để đánh giá mức độ oi bức, dễ tạo mây hoặc mưa.*

---

### 🌧️ Mưa, tuyết, nắng

| Tên biến | Giải thích | Đơn vị |
|----------|------------|--------|
| `prcp`   | **Lượng mưa (precipitation)** – tổng lượng mưa trong ngày | mm |
| `snow`   | Độ dày lớp tuyết (nếu có tuyết) | mm |
| `tsun`   | Thời lượng ánh nắng mặt trời trong ngày | phút |

---

### 🌬️ Gió & áp suất

| Tên biến | Giải thích | Đơn vị |
|----------|------------|--------|
| `wdir`   | **Hướng gió trung bình**, tính theo độ (0° là Bắc, 90° là Đông, 180° là Nam, 270° là Tây) | ° |
| `wspd`   | **Tốc độ gió trung bình** trong ngày | km/h |
| `wpgt`   | **Tốc độ gió lớn nhất (giật gió)** đo được trong ngày | km/h |
| `pres`   | Áp suất khí quyển trung bình | hPa (hectopascal) |

📌 *Áp suất khí quyển và gió là những yếu tố ảnh hưởng mạnh đến sự hình thành bão, mây, và thời tiết cực đoan.*

---

### ☁️ Mây

| Tên biến | Giải thích | Đơn vị |
|----------|------------|--------|
| `coco`   | **Mức độ che phủ mây (cloud cover)** – thường biểu diễn theo code 0 đến 8 hoặc phần trăm | (0–100%) hoặc mã code |

---

## ✅ Tóm tắt sử dụng:

| Loại | Thông số quan trọng |
|------|----------------------|
| Dự báo nhiệt độ | `temp`, `dwpt`, `rhum`, `pres` |
| Dự báo mưa | `prcp`, `rhum`, `pres`, `temp`, `coco` |
| Dự báo gió | `wspd`, `wpgt`, `wdir` |
| Dự báo độ nắng | `tsun`, `coco`, `pres`, `rhum` |

---


- Dùng các cột như `temp`, `rhum`, `wspd`, `pres`, `prcp`, `coco` làm **đặc trưng đầu vào (features)**
- Dự báo `temp`, `rhum`, `prcp`… trong 3–5 ngày tới

### Cào dữ liệu từ 2025-03-17 đến 2025-04-16

In [2]:
from datetime import datetime, timedelta
from meteostat import Point, Hourly
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time # Để thử lại geocoding

# --- Hàm lấy tọa độ từ tên thành phố ---
def get_coordinates(city_name, retries=3, delay=2):
    """
    Sử dụng Geopy để lấy tọa độ (latitude, longitude) từ tên thành phố.
    """
    geolocator = Nominatim(user_agent="weather_app_user_unique_id_hourly") # User-agent khác biệt một chút
    print(f"Đang tìm tọa độ cho '{city_name}'...")
    for attempt in range(retries):
        try:
            location = geolocator.geocode(city_name, timeout=10) # Tăng timeout một chút
            if location:
                print(f"Đã tìm thấy tọa độ: ({location.latitude}, {location.longitude})")
                return location.latitude, location.longitude
            else:
                # Thử tìm kiếm rộng hơn nếu không tìm thấy ngay
                location_feature = geolocator.geocode(city_name, exactly_one=False, limit=1, feature_type='city')
                if location_feature:
                     loc = location_feature[0]
                     print(f"Đã tìm thấy tọa độ (feature_type='city'): ({loc.latitude}, {loc.longitude})")
                     return loc.latitude, loc.longitude
                else:
                    print(f"Không tìm thấy tọa độ cho: {city_name}")
                    return None, None # Trả về None nếu không tìm thấy sau lần thử đầu
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Lỗi Geocoding (lần {attempt + 1}/{retries}): {e}. Đang thử lại sau {delay} giây...")
            time.sleep(delay)
        except Exception as e:
            print(f"Lỗi Geocoding không xác định: {e}")
            # Có thể log lỗi chi tiết ở đây nếu cần
            # import traceback
            # traceback.print_exc()
            return None, None # Trả về None nếu có lỗi khác
    print(f"Không thể lấy tọa độ cho {city_name} sau {retries} lần thử.")
    return None, None

# --- Hàm lấy dữ liệu thời tiết LỊCH SỬ THEO GIỜ từ Meteostat ---
def get_hourly_historical_weather(lat, lon, start_date, end_date):
    """
    Lấy dữ liệu thời tiết lịch sử THEO GIỜ từ Meteostat bằng tọa độ.
    """
    try:
        # Tạo điểm vị trí
        location = Point(lat, lon) # Có thể thêm độ cao nếu biết: Point(lat, lon, elevation)

        start_date_str = start_date.strftime('%Y-%m-%d')
        end_date_str = end_date.strftime('%Y-%m-%d')
        print(f"Đang lấy dữ liệu Meteostat HOURLY cho ({lat}, {lon}) từ {start_date_str} đến {end_date_str}...")

        # Sử dụng lớp Hourly
        data_fetcher = Hourly(location, start_date, end_date)

        # Lấy dữ liệu
        df = data_fetcher.fetch()

        if df.empty:
            print(f"Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat cho khoảng thời gian/vị trí này.")
            return None

        print(f"Đã lấy thành công {len(df)} bản ghi theo giờ từ Meteostat.")
        df['source'] = 'Meteostat' # Thêm cột nguồn dữ liệu
        return df

    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu từ Meteostat: {e}")
        # import traceback
        # traceback.print_exc() # In chi tiết lỗi để debug
        return None

# --- Hàm chính để chạy chương trình ---
if __name__ == "__main__":
    city = input("Nhập tên vị trí địa lý (ví dụ: Hà Nội, Ho Chi Minh City, London): ")
    latitude, longitude = get_coordinates(city)

    if latitude is None or longitude is None:
        print("Không thể tiếp tục vì không lấy được tọa độ.")
    else:
        # Tự động xác định khoảng thời gian: 30 ngày gần nhất
        end_date = datetime.now()
        start_date = end_date - timedelta(days=30) # Lấy 30 ngày trước đó

        # Định dạng ngày tháng để dùng trong tên file
        start_date_str_file = start_date.strftime('%Y%m%d')
        end_date_str_file = end_date.strftime('%Y%m%d')

        print(f"\n--- Bắt đầu lấy dữ liệu lịch sử theo giờ cho '{city}' ---")
        historical_weather_hourly = get_hourly_historical_weather(latitude, longitude, start_date, end_date)

        if historical_weather_hourly is not None and not historical_weather_hourly.empty:
            # Tự động lưu vào file CSV
            # Tạo tên file động
            safe_city_name = "".join(c if c.isalnum() else "_" for c in city) # Làm sạch tên thành phố cho tên file
            filename = f"hourly_weather_{safe_city_name}_{start_date_str_file}_to_{end_date_str_file}.csv"

            try:
                # Lưu DataFrame vào CSV, bao gồm cả index (là timestamp)
                historical_weather_hourly.to_csv(filename, index=True)
                print("-" * 30)
                print(f"ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: {filename}")
                print(f"Số dòng dữ liệu: {len(historical_weather_hourly)}")
                print("Một vài dòng dữ liệu đầu tiên:")
                print(historical_weather_hourly.head())
                print("-" * 30)
            except Exception as e:
                print(f"Lỗi khi lưu file CSV '{filename}': {e}")
        else:
            print("-" * 30)
            print("Không có dữ liệu lịch sử theo giờ để lưu.")
            print("-" * 30)

        print("Chương trình hoàn tất.")

Đang tìm tọa độ cho ''...
Lỗi Geocoding không xác định: Nominatim.geocode() got an unexpected keyword argument 'feature_type'
Không thể tiếp tục vì không lấy được tọa độ.


In [3]:
from datetime import datetime, timedelta
from meteostat import Point, Hourly
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time # Để thử lại geocoding và thêm delay
import os   # Để kiểm tra file tồn tại

# --- Tên file chứa danh sách tỉnh thành ---
PROVINCE_LIST_FILE = "danh_sach_tinh_thanh_khong_dau.txt"

# --- Hàm lấy tọa độ từ tên thành phố ---
def get_coordinates(city_name, retries=3, delay=2):
    """
    Sử dụng Geopy để lấy tọa độ (latitude, longitude) từ tên thành phố.
    """
    # Quan trọng: Sử dụng user_agent duy nhất và tuân thủ chính sách sử dụng của Nominatim
    geolocator = Nominatim(user_agent="vn_weather_fetcher_your_email@example.com") # Thay bằng email của bạn hoặc ID duy nhất
    print(f"[{city_name}] Đang tìm tọa độ...")
    for attempt in range(retries):
        try:
            # Thêm quốc gia để tăng độ chính xác cho tên tiếng Việt không dấu
            location = geolocator.geocode(f"{city_name}, Vietnam", timeout=15)
            if location:
                print(f"[{city_name}] Đã tìm thấy tọa độ: ({location.latitude:.4f}, {location.longitude:.4f})")
                return location.latitude, location.longitude
            else:
                # Thử lại không có ", Vietnam" nếu lần đầu thất bại
                print(f"[{city_name}] Không tìm thấy với ', Vietnam', thử lại chỉ với tên...")
                time.sleep(delay / 2) # Delay nhỏ trước khi thử lại
                location = geolocator.geocode(city_name, timeout=15)
                if location:
                    print(f"[{city_name}] Đã tìm thấy tọa độ (lần 2): ({location.latitude:.4f}, {location.longitude:.4f})")
                    return location.latitude, location.longitude
                else:
                    print(f"[{city_name}] Không tìm thấy tọa độ.")
                    return None, None # Trả về None nếu không tìm thấy
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"[{city_name}] Lỗi Geocoding (lần {attempt + 1}/{retries}): {e}. Đang thử lại sau {delay} giây...")
            time.sleep(delay)
        except Exception as e:
            print(f"[{city_name}] Lỗi Geocoding không xác định: {e}")
            return None, None
    print(f"[{city_name}] Không thể lấy tọa độ sau {retries} lần thử.")
    return None, None

# --- Hàm lấy dữ liệu thời tiết LỊCH SỬ THEO GIỜ từ Meteostat ---
def get_hourly_historical_weather(city_name, lat, lon, start_date, end_date):
    """
    Lấy dữ liệu thời tiết lịch sử THEO GIỜ từ Meteostat bằng tọa độ.
    """
    try:
        location = Point(lat, lon)
        start_date_str = start_date.strftime('%Y-%m-%d')
        end_date_str = end_date.strftime('%Y-%m-%d')
        print(f"[{city_name}] Đang lấy dữ liệu Meteostat HOURLY ({lat:.4f}, {lon:.4f}) từ {start_date_str} đến {end_date_str}...")

        data_fetcher = Hourly(location, start_date, end_date)
        df = data_fetcher.fetch()

        if df.empty:
            print(f"[{city_name}] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.")
            return None

        print(f"[{city_name}] Đã lấy thành công {len(df)} bản ghi theo giờ.")
        df['source'] = 'Meteostat'
        df['province'] = city_name # Thêm cột tên tỉnh/thành phố
        return df

    except Exception as e:
        print(f"[{city_name}] Lỗi khi lấy dữ liệu từ Meteostat: {e}")
        return None

# --- Hàm đọc danh sách tỉnh từ file ---
def read_province_list(filename):
    """Đọc danh sách tỉnh từ file text, bỏ qua dòng trống và loại bỏ khoảng trắng thừa."""
    if not os.path.exists(filename):
        print(f"Lỗi: Không tìm thấy file danh sách tỉnh: {filename}")
        return None
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            provinces = [line.strip() for line in f if line.strip()] # Đọc, loại bỏ khoảng trắng, bỏ qua dòng trống
        print(f"Đã đọc được {len(provinces)} tỉnh/thành từ file {filename}.")
        return provinces
    except Exception as e:
        print(f"Lỗi khi đọc file {filename}: {e}")
        return None

# --- Hàm chính để chạy chương trình ---
if __name__ == "__main__":
    provinces = read_province_list(PROVINCE_LIST_FILE)

    if provinces:
        # Xác định khoảng thời gian: 30 ngày gần nhất
        end_date = datetime.now()
        start_date = end_date - timedelta(days=30)
        start_date_str_file = start_date.strftime('%Y%m%d')
        end_date_str_file = end_date.strftime('%Y%m%d')

        # Tạo thư mục để lưu kết quả nếu chưa có
        output_dir = "weather_data_hourly"
        os.makedirs(output_dir, exist_ok=True)
        print(f"Dữ liệu sẽ được lưu vào thư mục: {output_dir}")

        total_provinces = len(provinces)
        success_count = 0
        fail_coord_count = 0
        fail_fetch_count = 0

        # Lặp qua từng tỉnh trong danh sách
        for i, city_name in enumerate(provinces):
            print(f"\n--- [{i+1}/{total_provinces}] Đang xử lý: {city_name} ---")

            latitude, longitude = get_coordinates(city_name)

            if latitude is None or longitude is None:
                print(f"[{city_name}] Bỏ qua do không lấy được tọa độ.")
                fail_coord_count += 1
                time.sleep(1) # Delay nhỏ trước khi xử lý tỉnh tiếp theo
                continue # Chuyển sang tỉnh tiếp theo

            # Delay giữa các lần gọi Geocoding để tránh bị block
            time.sleep(1.5) # Chờ 1.5 giây

            historical_weather_hourly = get_hourly_historical_weather(city_name, latitude, longitude, start_date, end_date)

            if historical_weather_hourly is not None and not historical_weather_hourly.empty:
                # Tạo tên file động và đường dẫn đầy đủ
                safe_city_name = "".join(c if c.isalnum() else "_" for c in city_name.replace(" ", "_"))
                filename = f"hourly_weather_{safe_city_name}_{start_date_str_file}_to_{end_date_str_file}.csv"
                filepath = os.path.join(output_dir, filename)

                try:
                    historical_weather_hourly.to_csv(filepath, index=True)
                    print(f"[{city_name}] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: {filepath}")
                    success_count += 1
                except Exception as e:
                    print(f"[{city_name}] Lỗi khi lưu file CSV '{filepath}': {e}")
                    # Có thể coi đây là một dạng lỗi fetch nếu lưu thất bại
                    fail_fetch_count +=1

            else:
                print(f"[{city_name}] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.")
                fail_fetch_count += 1

            # Delay giữa các lần xử lý tỉnh để tránh quá tải Meteostat (nếu cần)
            time.sleep(0.5) # Chờ nửa giây

        print("\n--- TỔNG KẾT ---")
        print(f"Tổng số tỉnh/thành cần xử lý: {total_provinces}")
        print(f"Số lượng xử lý và lưu thành công: {success_count}")
        print(f"Số lượng lỗi không tìm thấy tọa độ: {fail_coord_count}")
        print(f"Số lượng lỗi không lấy được/lưu dữ liệu thời tiết: {fail_fetch_count}")
        print("Chương trình hoàn tất.")

    else:
        print("Không có danh sách tỉnh để xử lý. Chương trình kết thúc.")

Đã đọc được 63 tỉnh/thành từ file danh_sach_tinh_thanh_khong_dau.txt.
Dữ liệu sẽ được lưu vào thư mục: weather_data_hourly

--- [1/63] Đang xử lý: An Giang ---
[An Giang] Đang tìm tọa độ...
[An Giang] Đã tìm thấy tọa độ: (10.5392, 105.2313)
[An Giang] Đang lấy dữ liệu Meteostat HOURLY (10.5392, 105.2313) từ 2025-03-18 đến 2025-04-17...
[An Giang] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[An Giang] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [2/63] Đang xử lý: Ba Ria - Vung Tau ---
[Ba Ria - Vung Tau] Đang tìm tọa độ...
[Ba Ria - Vung Tau] Đã tìm thấy tọa độ: (10.5739, 107.3284)
[Ba Ria - Vung Tau] Đang lấy dữ liệu Meteostat HOURLY (10.5739, 107.3284) từ 2025-03-18 đến 2025-04-17...
[Ba Ria - Vung Tau] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Ba Ria - Vung Tau] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [3/63] Đang xử lý: Bac Giang ---
[Bac Giang] Đang tìm tọa độ...
[Bac Giang] Đã tìm thấy tọa độ: (21.3170, 106.

In [1]:
from datetime import datetime, timedelta
from meteostat import Point, Hourly, Daily, Stations
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
import os
import warnings

# Bỏ qua cảnh báo
warnings.filterwarnings('ignore')

# --- Tên file chứa danh sách tỉnh thành ---
PROVINCE_LIST_FILE = "danh_sach_tinh_thanh_khong_dau.txt"

# Danh sách các thành phố lớn của Việt Nam để fallback
VIETNAM_MAJOR_CITIES = {
    "Hanoi": (21.0278, 105.8342),
    "Ho Chi Minh City": (10.8231, 106.6297),
    "Da Nang": (16.0544, 108.2022),
    "Hai Phong": (20.8449, 106.6881),
    "Can Tho": (10.0452, 105.7469),
    "Bien Hoa": (10.9574, 106.8426),
    "Hue": (16.4637, 107.5909),
    "Nha Trang": (12.2388, 109.1968),
    "Quy Nhon": (13.7829, 109.2196),  # Đại diện cho Bình Định
    "Vinh": (18.6734, 105.6819),      # Đại diện cho Nghệ An
    "Ha Long": (20.9581, 107.0949),   # Đại diện cho Quảng Ninh
}

# Ánh xạ các tỉnh nhỏ tới thành phố lớn
PROVINCE_TO_CITY_MAPPING = {
    "Binh Dinh": "Quy Nhon",
    "Nghe An": "Vinh",
    "Quang Ninh": "Ha Long",
    "Thanh Hoa": "Thanh Hoa",
    "Hai Duong": "Hai Phong",
    "Bac Ninh": "Hanoi",
    "Hung Yen": "Hanoi",
}

# --- Hàm lấy tọa độ từ tên thành phố với fallback ---
def get_coordinates(city_name, retries=3, delay=2):
    """
    Sử dụng Geopy để lấy tọa độ với hỗ trợ fallback cho các tỉnh nhỏ.
    """
    geolocator = Nominatim(user_agent=f"vn_weather_{city_name.lower().replace(' ', '_')}")
    print(f"[{city_name}] Đang tìm tọa độ...")
    
    # Kiểm tra xem có trong danh sách thành phố lớn không
    for key, (lat, lon) in VIETNAM_MAJOR_CITIES.items():
        if key.lower() == city_name.lower():
            print(f"[{city_name}] Đã tìm thấy tọa độ từ danh sách: ({lat:.4f}, {lon:.4f})")
            return lat, lon
    
    # Kiểm tra xem có trong ánh xạ tỉnh nhỏ không
    for province, major_city in PROVINCE_TO_CITY_MAPPING.items():
        if province.lower() == city_name.lower():
            if major_city in VIETNAM_MAJOR_CITIES:
                lat, lon = VIETNAM_MAJOR_CITIES[major_city]
                print(f"[{city_name}] Sử dụng tọa độ của {major_city}: ({lat:.4f}, {lon:.4f})")
                return lat, lon
    
    # Danh sách các phương pháp thử
    geocoding_methods = [
        lambda: geolocator.geocode(f"{city_name}, Vietnam", timeout=15),
        lambda: geolocator.geocode(city_name, timeout=15),
        lambda: geolocator.geocode(f"{city_name} Province, Vietnam", timeout=15),
        lambda: geolocator.geocode(f"{city_name} City, Vietnam", timeout=15)
    ]
    
    # Thử từng phương pháp
    for attempt in range(retries):
        for method_idx, method in enumerate(geocoding_methods):
            try:
                location = method()
                if location:
                    print(f"[{city_name}] Đã tìm thấy tọa độ (phương pháp {method_idx+1}): ({location.latitude:.4f}, {location.longitude:.4f})")
                    return location.latitude, location.longitude
            except (GeocoderTimedOut, GeocoderServiceError) as e:
                print(f"[{city_name}] Lỗi Geocoding (lần {attempt+1}/{retries}, phương pháp {method_idx+1}): {e}.")
                time.sleep(delay)
            except Exception as e:
                print(f"[{city_name}] Lỗi không xác định (phương pháp {method_idx+1}): {e}")
    
    # Nếu tất cả đều thất bại, trả về Hà Nội làm mặc định
    print(f"[{city_name}] Không thể lấy tọa độ. Sử dụng Hà Nội làm mặc định.")
    return VIETNAM_MAJOR_CITIES["Hanoi"]

# --- Hàm lấy dữ liệu thời tiết cho ngày hiện tại ---
def get_today_hourly_weather(city_name, lat, lon):
    """
    Lấy dữ liệu thời tiết theo giờ cho ngày hiện tại.
    """
    try:
        # Lấy ngày hiện tại
        today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        tomorrow = today + timedelta(days=1)
        
        # Định dạng ngày tháng
        today_str = today.strftime('%Y-%m-%d')
        
        print(f"[{city_name}] Đang lấy dữ liệu cho ngày {today_str}...")
        
        # Thử với bán kính tìm kiếm rộng hơn
        location = Point(lat, lon)
        data_fetcher = Hourly(location, today, tomorrow, radius=75000)  # Bán kính 75km
        
        df = data_fetcher.fetch()
        
        if df.empty:
            print(f"[{city_name}] Không tìm thấy dữ liệu theo giờ từ Meteostat cho ngày {today_str}.")
            
            # Thử tìm trạm thời tiết gần nhất
            print(f"[{city_name}] Đang tìm trạm thời tiết gần nhất...")
            stations = Stations()
            nearby_stations = stations.nearby(lat, lon, 100)  # Bán kính 100km
            nearby_stations = nearby_stations.fetch()
            
            if not nearby_stations.empty:
                station_id = nearby_stations.index[0]  # Lấy trạm đầu tiên
                station_name = nearby_stations.iloc[0]['name']
                print(f"[{city_name}] Tìm thấy trạm {station_name} (ID: {station_id}). Đang lấy dữ liệu...")
                
                data_fetcher = Hourly(station_id, today, tomorrow)
                df = data_fetcher.fetch()
                
                if not df.empty:
                    print(f"[{city_name}] Đã lấy thành công {len(df)} bản ghi từ trạm {station_name}.")
                    df['source'] = f'Meteostat_Station_{station_id}'
                    df['station_name'] = station_name
                    df['province'] = city_name
                    return df
                else:
                    print(f"[{city_name}] Không tìm thấy dữ liệu từ trạm {station_name}.")
            
            # Thử lấy dữ liệu theo ngày và tạo dữ liệu theo giờ
            print(f"[{city_name}] Thử lấy dữ liệu theo ngày và chuyển đổi...")
            daily_fetcher = Daily(location, today, tomorrow, radius=100000)  # Bán kính 100km
            daily_df = daily_fetcher.fetch()
            
            if not daily_df.empty:
                print(f"[{city_name}] Đã lấy được dữ liệu theo ngày. Đang chuyển đổi thành dữ liệu theo giờ...")
                
                # Tạo dữ liệu theo giờ từ dữ liệu theo ngày
                hourly_data = []
                for date, row in daily_df.iterrows():
                    current_hour = datetime.now().hour
                    # Tạo dữ liệu cho các giờ từ 0 đến giờ hiện tại
                    for hour in range(current_hour + 1):
                        time_point = today.replace(hour=hour)
                        hourly_data.append({
                            'time': time_point,
                            'temp': row['tavg'],
                            'dwpt': None,
                            'rhum': row.get('rhum'),
                            'prcp': row.get('prcp')/24 if row.get('prcp') is not None else None,
                            'wspd': row.get('wspd'),
                            'wdir': row.get('wdir'),
                            'pres': row.get('pres'),
                            'coco': row.get('coco'),
                            'source': 'Meteostat_Daily_Converted',
                            'province': city_name,
                            'is_converted': True
                        })
                
                if hourly_data:
                    df = pd.DataFrame(hourly_data)
                    df.set_index('time', inplace=True)
                    print(f"[{city_name}] Đã tạo {len(df)} bản ghi theo giờ từ dữ liệu theo ngày.")
                    return df
                
            print(f"[{city_name}] Không thể lấy dữ liệu thời tiết cho ngày hôm nay.")
            return None
        
        print(f"[{city_name}] Đã lấy thành công {len(df)} bản ghi theo giờ cho ngày {today_str}.")
        df['source'] = 'Meteostat'
        df['province'] = city_name
        return df
    
    except Exception as e:
        print(f"[{city_name}] Lỗi khi lấy dữ liệu thời tiết: {e}")
        return None

# --- Hàm đọc danh sách tỉnh từ file ---
def read_province_list(filename):
    """Đọc danh sách tỉnh từ file text."""
    if not os.path.exists(filename):
        print(f"Lỗi: Không tìm thấy file danh sách tỉnh: {filename}")
        return None
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            provinces = [line.strip() for line in f if line.strip()]
        print(f"Đã đọc được {len(provinces)} tỉnh/thành từ file {filename}.")
        return provinces
    except Exception as e:
        print(f"Lỗi khi đọc file {filename}: {e}")
        return None

# --- Hàm chính để chạy chương trình ---
if __name__ == "__main__":
    # Kiểm tra file danh sách tỉnh
    if not os.path.exists(PROVINCE_LIST_FILE):
        print(f"Tạo file danh sách tỉnh mẫu: {PROVINCE_LIST_FILE}")
        sample_provinces = [
            "Ha Noi", "Ho Chi Minh City", "Da Nang", "Hai Phong", "Can Tho",
            "Binh Dinh", "Hue", "Nha Trang", "Vinh", "Ha Long"
        ]
        try:
            with open(PROVINCE_LIST_FILE, 'w', encoding='utf-8') as f:
                f.write('\n'.join(sample_provinces))
            print(f"Đã tạo file danh sách tỉnh mẫu với {len(sample_provinces)} tỉnh/thành.")
        except Exception as e:
            print(f"Lỗi khi tạo file danh sách tỉnh mẫu: {e}")
    
    provinces = read_province_list(PROVINCE_LIST_FILE)

    if provinces:
        # Tạo thư mục để lưu kết quả
        today_str = datetime.now().strftime('%Y%m%d')
        output_dir = f"weather_today_{today_str}"
        os.makedirs(output_dir, exist_ok=True)
        print(f"Dữ liệu sẽ được lưu vào thư mục: {output_dir}")

        total_provinces = len(provinces)
        success_count = 0
        fail_count = 0

        # Lặp qua từng tỉnh trong danh sách
        for i, city_name in enumerate(provinces):
            print(f"\n--- [{i+1}/{total_provinces}] Đang xử lý: {city_name} ---")

            # Lấy tọa độ
            latitude, longitude = get_coordinates(city_name)
            
            # Delay giữa các lần gọi API
            time.sleep(1.5)

            # Lấy dữ liệu thời tiết
            weather_data = get_today_hourly_weather(city_name, latitude, longitude)

            if weather_data is not None and not weather_data.empty:
                # Tạo tên file
                safe_city_name = "".join(c if c.isalnum() else "_" for c in city_name.replace(" ", "_"))
                filename = f"today_weather_{safe_city_name}_{today_str}.csv"
                filepath = os.path.join(output_dir, filename)

                try:
                    # Lưu file
                    weather_data.to_csv(filepath)
                    print(f"[{city_name}] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: {filepath}")
                    
                    # Thống kê ngắn gọn
                    hour_count = len(weather_data)
                    temp_avg = weather_data['temp'].mean() if 'temp' in weather_data.columns else None
                    humid_avg = weather_data['rhum'].mean() if 'rhum' in weather_data.columns else None
                    
                    print(f"[{city_name}] Thống kê: {hour_count} giờ, Nhiệt độ TB: {temp_avg:.1f}°C, Độ ẩm TB: {humid_avg:.1f}%")
                    success_count += 1
                except Exception as e:
                    print(f"[{city_name}] Lỗi khi lưu file CSV: {e}")
                    fail_count += 1
            else:
                print(f"[{city_name}] Không lấy được dữ liệu thời tiết.")
                fail_count += 1

            # Delay để tránh quá tải API
            time.sleep(0.5)

        print("\n--- TỔNG KẾT ---")
        print(f"Tổng số tỉnh/thành đã xử lý: {total_provinces}")
        print(f"Số lượng xử lý thành công: {success_count}")
        print(f"Số lượng thất bại: {fail_count}")
        print(f"Dữ liệu đã được lưu trong thư mục: {output_dir}")
        print("Chương trình hoàn tất.")

    else:
        print("Không có danh sách tỉnh để xử lý. Vui lòng tạo file danh sách tỉnh và thử lại.")

Đã đọc được 63 tỉnh/thành từ file danh_sach_tinh_thanh_khong_dau.txt.
Dữ liệu sẽ được lưu vào thư mục: weather_today_20250417

--- [1/63] Đang xử lý: An Giang ---
[An Giang] Đang tìm tọa độ...
[An Giang] Đã tìm thấy tọa độ (phương pháp 1): (10.5392, 105.2313)
[An Giang] Đang lấy dữ liệu cho ngày 2025-04-17...
[An Giang] Lỗi khi lấy dữ liệu thời tiết: Hourly.__init__() got an unexpected keyword argument 'radius'
[An Giang] Không lấy được dữ liệu thời tiết.

--- [2/63] Đang xử lý: Ba Ria - Vung Tau ---
[Ba Ria - Vung Tau] Đang tìm tọa độ...
[Ba Ria - Vung Tau] Đã tìm thấy tọa độ (phương pháp 1): (10.5739, 107.3284)
[Ba Ria - Vung Tau] Đang lấy dữ liệu cho ngày 2025-04-17...
[Ba Ria - Vung Tau] Lỗi khi lấy dữ liệu thời tiết: Hourly.__init__() got an unexpected keyword argument 'radius'
[Ba Ria - Vung Tau] Không lấy được dữ liệu thời tiết.

--- [3/63] Đang xử lý: Bac Giang ---
[Bac Giang] Đang tìm tọa độ...
[Bac Giang] Đã tìm thấy tọa độ (phương pháp 1): (21.3170, 106.4380)
[Bac Giang] Đan